# Feature Extraction and Generate Vectors

In [ ]:
#load lib
import os
import string
import pandas as pd
from stopwords import get_stopwords
from hazm import word_tokenize,sent_tokenize
from hazm import POSTagger,Normalizer
import re
from sklearn.decomposition import PCA
stopwords = get_stopwords('persian')

In [ ]:
fa_short_words = ['و','که','را','از','به','با','در','تا','این','آن','است','بود']
fa_rabt_words = ['و','که','هم','چون','اگر','با','یا','تا','پس','چه','اما','ولی']
fa_char = []

In [ ]:
lst_vectors1 = []
lst_vectors2 = []
lst_vectors3 = []
for i in range(1,41):
    text_char_count = 0
    for j in range(1,6):
        path = 'Dataset-Text/{0}/{1}-{2}.txt'.format(i,i,j)
        if (os.path.isfile(path)):
            f = open('Dataset-Text/{0}/{1}-{2}.txt'.format(i,i,j),"r",encoding="utf-8")
            text = f.read()
            
            #lst_vectors2.append(stCounting(text)) # حروف ربط
            
            #lst_vectors3.append(CountTxtTags(text)) # تگ کلمات
            
            # فراوانی توزیع حروف
            tokens = word_tokenize(text)
            filtered_text = [word for word in tokens if word not in stopwords]
            text = ' '.join(filtered_text)
            text_char_count = len(text.replace(' ','').replace('\u200c','').translate(str.maketrans('', '', string.punctuation)).replace('،','').replace('«','').replace('»',''))
            lst_vectors1.append(count_farsi_letters(text,i,text_char_count))
            

In [ ]:
# Extracting the distribution of letters

farsi_letters = "ابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی"

def count_farsi_letters(text,aut_name,text_char_count):
    letter_count = []
    letter_count.append(aut_name)
    for letter in farsi_letters:
        if letter in text:
            letter_count.append(int((text.count(letter)/text_char_count) * 10 ** 3) / 10 ** 3)
            #letter_count.append(int((text.count(letter)/text_char_count) * 10 ** 2) / 10 ** 2)
        else:
            letter_count.append(0)
    return letter_count
    #print(letter_count)

In [ ]:
# Extracting the distribution of conjunctions - حروف ربط

word_count = []
txt_total_st_count = 0
def stCounting(txt):
    word_count = []
    txt_total_st_count = 0
    txt_token = txt.split(None)
    for wsw in fa_rabt_words:
        c = txt_token.count(wsw)
        txt_total_st_count = txt_total_st_count + c
    for wsw in fa_rabt_words:
        st = txt_token.count(wsw)
        if (st > 0):
            word_count.append(int((st / txt_total_st_count) * 10 ** 3) / 10 ** 3)
        else:
            word_count.append(0)
    return word_count

In [ ]:
# Extracting the distribution of pos-tags

lst_tag_count_all =[]
normalizer = Normalizer()
model_path = 'pos_tagger.model'
tagger = POSTagger(model = model_path)

def CountTxtTags(txt):
    normalize_text = normalizer.normalize(txt)
    tokenize_text = [word_tokenize(txt) for txt in sent_tokenize(normalize_text)]
    token_tag_list = tagger.tag_sents(tokenize_text)
    ADP=0
    NOUN=0 # اسم
    CCONJ=0 # حرف ربط
    ADJ=0 # صفت
    PUNCT=0 # علائم نگارشی
    ADV=0
    VERB=0 #فعل
    PRON=0 #ضمیر
    SCONJ=0 #
    all_tag_count = 0
    lst_tag_count =[]
    for sent in token_tag_list:
        for tag in sent:
            #print(tag[1])
            if (tag[1]=='ADP'): 
                ADP = ADP +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='NOUN'): 
                NOUN = NOUN +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='CCONJ'): 
                CCONJ = CCONJ +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='ADJ'): 
                ADJ = ADJ +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='PUNCT'): 
                PUNCT = PUNCT +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='ADV'): 
                ADV = ADV +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='VERB'): 
                VERB = VERB +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='PRON'): 
                PRON = PRON +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='SCONJ'): 
                SCONJ = SCONJ +1
                all_tag_count = all_tag_count + 1
    ADP = (int((ADP / all_tag_count) * 10 ** 3) / 10 ** 3)
    NOUN = (int((NOUN / all_tag_count) * 10 ** 3) / 10 ** 3)
    CCONJ = (int((CCONJ / all_tag_count) * 10 ** 3) / 10 ** 3)
    ADJ = (int((ADJ / all_tag_count) * 10 ** 3) / 10 ** 3)
    PUNCT = (int((PUNCT / all_tag_count) * 10 ** 3) / 10 ** 3)
    ADV = (int((ADV / all_tag_count) * 10 ** 3) / 10 ** 3)
    VERB = (int((VERB / all_tag_count) * 10 ** 3) / 10 ** 3)
    PRON = (int((PRON / all_tag_count) * 10 ** 3) / 10 ** 3)
    SCONJ = (int((SCONJ / all_tag_count) * 10 ** 3) / 10 ** 3)
    lst_tag_count=[ADP,NOUN,CCONJ,ADJ,PUNCT,ADV,VERB,PRON,SCONJ]
    #lst_tag_count_all.append(lst_tag_count)
    #print(all_tag_count)
    return lst_tag_count

In [ ]:
# Save vectors to excel files

df = pd.DataFrame(lst_vectors1) # convert list to dataframe

writer = pd.ExcelWriter('Vec.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='autors vector', index=False)
writer.save()
writer.close()